In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
%matplotlib inline

In [4]:
diab = pd.read_csv('pima-indians-diabetes.csv')

In [5]:
diab.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [6]:
diab.columns

Index(['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree', 'Age', 'Class', 'Group'],
      dtype='object')

In [7]:
col_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']
diab[col_to_norm] = diab[col_to_norm].apply(lambda x:(x - x.min())/(x.max() - x.min()))

In [8]:
diab.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [11]:
#create categorical col
num_preg = tf.feature_column.numeric_column('Number_pregnant')
glu_conc = tf.feature_column.numeric_column('Glucose_concentration')
bld_pres = tf.feature_column.numeric_column('Blood_pressure')
trcp = tf.feature_column.numeric_column('Triceps')
Inln = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
ped = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')
#create categorical col of group
assign_grp = tf.feature_column.categorical_column_with_hash_bucket('Group',hash_bucket_size=5)
#create bucketized column 
age_grp = tf.feature_column.bucketized_column(age,boundaries=[20,40,60,80])


In [23]:
x_data = diab.drop('Class',axis=1)
feat_col = [num_preg,glu_conc,bld_pres,trcp,Inln,bmi,ped,age_grp,assign_grp]

In [24]:
labels = diab['Class']

In [25]:
x_train,x_rest,y_train,y_rest = train_test_split(x_data,labels,test_size=0.5,random_state=101)
x_eval,x_test,y_eval,y_test = train_test_split(x_rest,y_rest,test_size=0.25,random_state=101)

In [26]:
#create input function for estimator
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=8,num_epochs=1000,shuffle=True)
#create model to train
est = tf.estimator.LinearClassifier(feature_columns=feat_col,n_classes=2)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/v4/yg76j28j56g98q8k2zlpb_lc0000gn/T/tmp_3lm47vg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a15e17320>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
est.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/v4/yg76j28j56g98q8k2zlpb_lc0000gn/T/tmp_3lm47vg/model.ckpt.
INFO:tensorflow:loss = 5.54518, step = 1
INFO:tensorflow:global_step/sec: 257.149
INFO:tensorflow:loss = 5.09697, step = 101 (0.391 sec)
INFO:tensorflow:global_step/sec: 387.641
INFO:tensorflow:loss = 5.09387, step = 201 (0.264 sec)
INFO:tensorflow:global_step/sec: 275.8
INFO:tensorflow:loss = 2.73896, step = 301 (0.365 sec)
INFO:tensorflow:global_step/sec: 284.784
INFO:tensorflow:loss = 4.56639, step = 401 (0.349 sec)
INFO:tensorflow:global_step/sec: 310.825
INFO:tensorflow:loss = 3.39946, step = 501 (0.326 sec)
INFO:tensorflow:global_step/sec: 340.154
INFO:tensorflow:loss = 4.7491, step = 601 (0.302 sec)
INFO:tensorflow:global_step/sec: 311.351
INFO:tensorflow:loss = 4.23012, step = 701 (0.303 sec)
INFO:tensorflow:global_step/sec: 270.289
INFO:tensorflow:loss = 4.39856, step = 801 (0.369 sec)
INFO:tensorflow:global_step/sec

In [28]:
#create evaluation funciton
eval_func = tf.estimator.inputs.pandas_input_fn(x=x_eval,y=y_eval,batch_size=8,num_epochs=1,shuffle=False)

est.evaluate(input_fn=eval_func,steps=1000)

INFO:tensorflow:Starting evaluation at 2018-04-15-04:28:06
INFO:tensorflow:Restoring parameters from /var/folders/v4/yg76j28j56g98q8k2zlpb_lc0000gn/T/tmp_3lm47vg/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-15-04:28:07
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.708333, accuracy_baseline = 0.677083, auc = 0.764957, auc_precision_recall = 0.588616, average_loss = 0.551437, global_step = 1000, label/mean = 0.322917, loss = 4.4115, prediction/mean = 0.380529


{'accuracy': 0.70833331,
 'accuracy_baseline': 0.67708337,
 'auc': 0.76495731,
 'auc_precision_recall': 0.58861554,
 'average_loss': 0.55143696,
 'global_step': 1000,
 'label/mean': 0.32291666,
 'loss': 4.4114957,
 'prediction/mean': 0.38052914}

In [30]:
#create prediction funciton
predict_func = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=8,num_epochs=1,shuffle=False)


In [31]:
#predict the results
pred = est.predict(predict_func)

In [32]:
results = list(pred)

INFO:tensorflow:Restoring parameters from /var/folders/v4/yg76j28j56g98q8k2zlpb_lc0000gn/T/tmp_3lm47vg/model.ckpt-1000


In [33]:
print(results)

[{'logits': array([-0.70664811], dtype=float32), 'logistic': array([ 0.33033991], dtype=float32), 'probabilities': array([ 0.66966009,  0.33033991], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([ 0.44725204], dtype=float32), 'logistic': array([ 0.60998571], dtype=float32), 'probabilities': array([ 0.39001432,  0.60998571], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object)}, {'logits': array([-0.74693751], dtype=float32), 'logistic': array([ 0.32148898], dtype=float32), 'probabilities': array([ 0.67851102,  0.32148895], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([-0.88298893], dtype=float32), 'logistic': array([ 0.29255879], dtype=float32), 'probabilities': array([ 0.70744121,  0.29255879], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object)}, {'logits': array([-0.44306153], dtype=float32), 'logistic': array([ 0.39101171]

In [34]:
results

[{'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.33033991], dtype=float32),
  'logits': array([-0.70664811], dtype=float32),
  'probabilities': array([ 0.66966009,  0.33033991], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.60998571], dtype=float32),
  'logits': array([ 0.44725204], dtype=float32),
  'probabilities': array([ 0.39001432,  0.60998571], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.32148898], dtype=float32),
  'logits': array([-0.74693751], dtype=float32),
  'probabilities': array([ 0.67851102,  0.32148895], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.29255879], dtype=float32),
  'logits': array([-0.88298893], dtype=float32),
  'probabilities': array([ 0.70744121,  0.29255879], dtype=float32)},
 {'class_ids': array([0]),
  'classes': arra